# Создание функции по определению полноты данных запроса пользователя 

# 1 Постановка задачи

Существуют такие случаи, когда пользователь отправляет запрос в тех.поддержку, однако данных для определения проблемы недостаточно. Поэтому нам необходимо отправлять пользователю письмо с просьбой дополнить свой ответ. С этой задачей могут справиться большие языковые модели.

# 2. Настройка окружения

Для того, чтобы решить данную задачу, импортируем необходимые библиотеки (open source) для работы, а также укажем, какую роль данные библиотеки играют в ходе исследования:
- **pandas**: используется для тестирования на реальных данных;
- **langchain**: используется для формирования промпта для модели;
- **langchain_ollama**: используется для доступа к локально равзернутым open source моделям.

In [1]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import ChatPromptTemplate
import pandas as pd

# 3. Выбор модели

Для выполнения задачи была выбрана LLM gemma2:9b, т.к. она была обучена на технических данных и способна хорошо понимать запросы пользователей, что является очень важным фактором для нашей задачи.

In [2]:
#base_url - путь к развернутой модели.

llm = OllamaLLM(
        model='gemma2',
        base_url='http://26.251.162.207:11434',
        temperature=0,
        format='json',
        num_ctx=8192,
        num_predict=128,
    )

# 4. Создание промпта

Внутри промпта мы обязательно задаем контекст того кем является LLM, в нашем случае это специалист пользовательской поддержки.

Затем мы сообщаем модели, что ей пришло письмо: оно состоит из заголовка(title) и текста письма(body).

Затем мы задаем модели задачу: оценивать, достаточно ли в письме информации, чтобы узнать тип оборудования(type_of_equipment) и точку отказа(fail_point), а также не обращала внимание на наименование оборудований(это у нас отдельная задача),

Далее сообщаем формат ответа json с полями flag и answer:
 
flag: true означает, что можно определить всю информацию о запросе, а false - что нет. В случае flag: false модель также выдает непустой answer с просьбой об уточнении информации о проблеме.

Далее выдаем инструкцию о том, чтобы модель отвечала кратко, а также задаем более точно формат ответа.

In [3]:
template = """Представь, что ты помощник специалиста пользовательской поддержки компании по производству и обслуживанию технического оборудования.

Тебе поступило письмо по электронной почте от клиента.

Заголовок письма: {title}.

Текст письма: {body}.

Проанализируй, достаточно ли информации из письма для того, чтобы подтвердить предоложение, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}. 

Отсутствия модели оборудования в сообщении не является признаком недостаточности информации. НИКОГДА не спрашивай о ней клиентов.

Ответ напиши в виде json-файла, в котором будет поле flag и поле answer.

Если ты считаешь, что информации достаточно для того, чтобы понять, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}, тогда в поле flag напиши true, а поле answer сделай пустым.

Если ты считаешь, что информации не достаточно для того, чтобы понять, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}, тогда в поле flag напиши false, а в поле answer напиши с точки зрения оператора поддержки, что необходимо добавить в письме, чтобы оно стало понятным.
Отвечай так, как если бы ты лично общалась с клиентом. Будь вежливой, поздоровайся с ним.

Следуй инструкциям, отвечай кратко. В ответе кроме json-файла ничего писать не нужно.

Структура ответа:

{{
  "flag": ...,
  "answer": ...
}}
"""

# 5. Чтение данных

С помощью библиотеки pandas прочитаем очищенные данные.

In [4]:
df = pd.read_csv('..\data\clear_data.csv')
df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HONOR\AppData\Local\Temp\ipykernel_10884\3412834767.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('..\data\clear_data.csv')


,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,+BB0EPgRDBEIEMQRDBDo- +BCEEGAQbBBA- +BB0EGg-2-...,Здравствуйте.\r\n\r\n Сломался ...,Ноутбук,Блок питания,C222090381
1,/ шумит кулер / отклеились ножки,"Коллеги, добрый день.\r\n\r\nПрошу осуществить...",Ноутбук,Вентилятор,C223010310
2,10552. СИЛА HK-1404 // замена блоков питания //,"Добрый день! Не работают блоки питания, прошу ...",Ноутбук,Блок питания,"C223010423, C223011174"
3,10979. Ноутбук СИЛА НК2-1404 // не включается,Добрый день! Ноутбук не включается,Ноутбук,Материнская плата,C223012335
4,BIOS'ы,"Здравствуйте!\r\nПоделитесь, пожалуйста, после...",Ноутбук,Консультация,C223010345
...,...,...,...,...,...
180,по неисправной плашке памяти на dd203 - Enclos...,Добрый день!\r\nПросьба открыть заявку по неис...,СХД,Оперативная память,CKM00193800300
181,полосы на матрице НК2-3404,Полосы на матрице.,Ноутбук,Матрица,C223090893
182,полосы на экране и отклеились ножки,Переодически (было 3-4 раза) появляются полосы...,Ноутбук,Матрица,c223012503
183,ремонт ноутбука СИЛА НК2-1404,не работает порт usd и порт type-c на ноутбуке...,Ноутбук,Материнская плата,C223100081


# 6. Создание тестирующей функции

Создадим тестирующую функцию, которая принимает dataframe, index строки, шаблон промпта и LLM для выдачи результата работы модели.

In [5]:
def check_prompt(df, index, prompt_template=template, llm=llm):
    title = df.iloc[index]['Тема']
    body = df.iloc[index]['Описание']
    fail_point = df.iloc[index]['Точка отказа']
    type_of_equipment = df.iloc[index]['Тип оборудования']


    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain_ = prompt | llm 


    return chain_.invoke({
            'title': title,
            'body': body,
            'type_of_equipment': type_of_equipment,
            'fail_point': fail_point,
        }), title, body



Пример работы модели.

In [6]:
answer, title, body = check_prompt(df, 3, prompt_template=template, llm=llm)
print(answer)
print(title)
print(body)

{
  "flag": false,
  "answer": "Здравствуйте! Чтобы лучше понять ситуацию, пожалуйста, расскажите подробнее о том, что происходит с ноутбуком. Например, слышите ли вы какие-либо звуки при включении? Может быть, индикаторы загораются?"
} 

10979. Ноутбук СИЛА НК2-1404 // не включается
Добрый день! Ноутбук не включается
